# Practical Introduction to Statistics

This notebook provides a set of exercises and along with discussions and explanations to supplement the experiments we will conduct throughout the course. **The goal of this first workshop is to get you comfortable with the basics of data exploration, statistical modeling, and quantifying uncertainties.** Next time, we will explore more details about how we can make inferences from our data using more complicated models and how to numerically implement them.

Throughout this notebook you may encounter particular portions/exercises that you may not know how to do off the top of your head. This is normal and happens all the time in research, since you're often learning as much code as you need to get by for any particular project. If this happens, remember that Google, Stack Exchange forums, and package documentation sites are your friends! Please don't be afraid to fill up reams of tabs with searches.

# Preamble

Whenever you're coding, it's always a good idea to try and import any expected dependencies early on. I've included a few standard ones along with their typical aliases, as well as a few to guarantee compatibility between Python 2 and 3 (so it doesn't matter which one you're running).

This might require installing a few packages. If so, try:

`` conda install <package> ``

or 

``pip install package``

and reloading the notebook.

In [ ]:
# Python 2/3 compatibility
from __future__ import print_function, division
from six.moves import range

# in case we want to call the terminal (e.g., `rm`)
import sys
import os

# numerics
import numpy as np

# science-oriented utility functions
import scipy

# plotting
import matplotlib
from matplotlib import pyplot as plt

# plot within the notebook instead of new windows
%matplotlib inline

We'll import some additional packages later, but it's always best practice to put them as early as possible so that you know right away if something isn't available when running your code.

# Data Collection

It's important to get some practice with the data collection process, so at this point we will be looking at a modeling the temperature calibration of a simple receiver. Using the setup in class, fill in the data table below.

In [ ]:
# temperature
T_hot = ...  # room temperature
T_cold = ...  # cooled with liquid nitrogen

# Power measurements
P_hot = ...  # array of measured power
P_cold = ...  # array of measured power

We also want to collect some extra observations (which we will use later). Please add those in here.

In [ ]:
# extra power measurements
P_hot_2 = ...
P_cold_2 = ...

# Data Exploration

These observations define a set of $y$ values for two measured $x$ values, which we can use to derive a linear fit for the temperature. This is enough to get us started on the basics on statistical analysis.

Before getting really quantitative, let's take a moment to examine our data in a bit more detail. Below, plot the data using `plt.plot` with the temperature on the x-axis and power on the y-axis. **Please make sure to add axis labels and a title and note appropriate units.**

In [ ]:
# plot power vs temperature
fig = plt.figure(figsize=(10, 10))  # create a figure
plt.plot(...)

# "prettify" by adding labels

# finalize
plt.tight_layout()  # helps keep things neat

One of the first things that you can do when fitting data is a process known as **"chi by eye"**...which is a cheeky way of saying just play around with the parameters until you get a fit that looks decent. You'd be surprised how close this can often get you to something more rigorous.

**Add in a linear fit to the above plot following**

$$ y = ax + b $$

where $a$ is the slope and $b$ is the intercept. What does this relationship broadly imply? How well could we expect to do at $T=0$?

# Mean and Standard Deviation

While the above method might get us close, it is by no means a rigorous result. To make this more robust, we need to quantify how the uncertainties in our measurements propagate to the uncertainties in our results.

One of the bedrock ways of quantifying uncertainty is to assume that the data are randomly generated from a **Normal** (Gaussian) distribution with some mean $\mu$ and standard deviation $\sigma$. It can be shown that the best guess for the mean and standard deviation from a set of noisy observations is

$$ \mu = \frac{1}{n} \sum_{i=1}^{n} y_i $$ 

and 

$$ \sigma = \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (y_i - \mu)^2} $$

**Compute the mean and standard deviation for the measured powers "manually" and using convenience functions in `numpy`.**

In [ ]:
# direct implementation
n = ...  # number of objects
P_hot_mean = ...
P_hot_std = ...
P_cold_mean = ...
P_cold_std = ...

# with convenience functions
P_hot_mean2 = ...
P_hot_std2 = ...
P_cold_mean2 = ...
P_cold_std2 = ...

Note there's a slight disagreement between the value from `numpy` and the analytic solution, which is based upon whether we divide by $n$ or $n-1$. The former case is more precise but biased, while the latter is less precise but unbiased. We will just multiply our result by $\sqrt{(n-1)/n}$ for now to stay consistent.

In [ ]:
P_hot_std *= np.sqrt((n - 1) / n)
P_cold_std *= np.sqrt((n - 1) / n)

These values are one of the ways in which astronomers try to quantify uncertainties. In other words, we are making a statement here about how likely it is that we would expect to see a new measurement $y_{\rm new}$ based on our current measurements $y$ assuming our measurements follow the assumptions we made. Let's explore a few ways we can quantify this.

# N-sigma deviations and Confidence Intervals

One common way a lot of researchers talk about uncertainties is talking about "how many sigma" an observation is away from the expected value. This can give a rough estimate of how likely/unlikely we would be to observed it.

The cell below shows what this looks like for a few sigma. See if you can parse through each segment of the code to broadly understand what's going on.

In [ ]:
# generate normal PDF, i.e. probability of
# observation as a function of sigma deviations
# away from the mean
from scipy import stats  # statistics utilities
x = np.linspace(-5, 5, 1000)
y = stats.norm.pdf(x)

# plot N-sigma intervals
plt.figure(figsize=(12, 6))
plt.plot(x, y, lw=3, color='blue')
for sigma in [1, 2, 3, 4, 5]:
    plt.fill_between(x, y, where=(np.abs(x) <= sigma), 
                     alpha=0.3/sigma, color='blue')
plt.xticks(np.arange(-5, 6))
plt.xlabel('Sigma')
plt.ylabel('Probability of Measurement')
plt.title('Gaussian')
plt.tight_layout()

As the above plot shows, each $\pm N \sigma$ bound corresponds to some portion of the total probability of observing the next measurement. To put this another way, there is an $X$% chance that the next observation will be between $\pm N$ standard deviations from the mean. This defines what is known as a **Confidence Interval**, which says how "confident" we are that the next observation will be between $\mu - N\sigma$ and $\mu + N\sigma$.

Using your knowledge of Gaussians and/or Google, **define the 2-sigma 95% confidence intervals for the two temperature measurements** (i.e. 95% of observations are within $\pm 2\sigma$).

In [ ]:
# define confidence intervals for P_hot
...

# define confidence intervals for P_cold
...

Let's verify this result by using some numerical simulation by generating random data using `numpy.random`. We should hopefully see that the number of observations within our $N$-sigma bounds agree with what we'd expect from what we computed above.

In [ ]:
# generate normally-distributed random numbers
n_rand = 100000
P_hot_rand = np.random.normal(...)  # random realizations of P_hot
P_cold_rand = np.random.normal(...)  # random realizations of P_cold

# check fraction of observations within confidence intervals
# sketch of example below
n_hot_2sig = np.sum((P_hot_rand >= P_2siglow) & 
                     (P_hot_rand < P_2sighot))  # select within 1-sigma
f_hot_2sig = n_hot_2sig / n_rand  # compute fraction
print(f_hot_2sig, 0.954499736103642)  # print result vs truth

# p-values and Hypothesis Testing

Another way to look at the result from above is in terms of **hypothesis testing**. In our calculations above, we have assumed that our model for the data (using the mean $\mu$ and standard deviation $\sigma$) is correct, and use that model to make predictions about what we think the next data will look like. But what if our model is wrong? One way to find out is to compare how well the next measurement agrees with our model. If it's extremely improbable under our current hypothesis, then our hypothesis is likely wrong and we need to update our model.

Our way to quantify this is using **p-values**, which is the probability that you would see an observation *at least as extreme* as the one you observe. This is just the flip-side of the confidence interval. An illustration is shown below.

In [ ]:
# plot p-value example
obs = 1.7  # offset (in sigma) of measurement from mean
plt.figure(figsize=(12, 6))
plt.plot(x, y, lw=3, color='blue')
plt.vlines(obs, 0, max(y), colors='red', lw=2)
plt.fill_between(x, y, where=np.abs(x) >= abs(obs), 
                 alpha=0.6, color='blue')
plt.xticks(np.arange(-5, 6))
plt.xlabel('Sigma')
plt.ylabel('Probability of Measurement')
plt.title('Gaussian')
plt.tight_layout()

If the p-value is small enough, then we should reject our current model and try to pick something different. Typical values include $p=0.05$, $p=0.01$, and $p=0.001$. I personally like the latter since I'm very conservative about trying to assume new models without ample evidence for them.

At this point, we now want to check whether our extra observations from earlier imply that our assumptions for the mean and standard deviations for $P_{\rm hot}$ and $P_{\rm cold}$ are justified.

**Compute the corresponding p-values for the extra observations above** by computing the fraction of simulated power measurements from earlier whose deviations from the mean exceed the deviation of the measured values.

In [ ]:
# define normalized residuals
P_hot_rand_sdev = np.abs(P_hot_rand - P_hot_mean) / P_hot_std
P_cold_rand_sdev = np.abs(P_cold_rand - P_cold_mean) / P_cold_std

# p-values for extra P_hot measurement (pick one)
...

# p-values for extra P_cold measurement (pick one)
...

If you have time and are interested in doing this more precisely, see if you can figure out how to compute the exact values from `scipy.stats.norm` using the [online documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm).

# Examining Uncertainties

Now that we better understand the uncertainties from our measurement (and the limitations of our current approximations), we now want to figure out how to propagate those errors into our temperature calibration. As a first step, plot the noisy data using `plt.errorbar`. Feel free to play around with the options to get the results to look nice, and always remember to label your axes and plots!

In [ ]:
# plot power vs temperature with errors
fig = plt.figure(figsize=(10, 10))
plt.errorbar(...)
plt.tight_layout()

# Best-fit Line

At this point, we're going to go through several iterations of propagating our errors into our results, based on a few different methods. First, we see that the slope of a line from two datapoints is just

$$ a = \frac{P_{\rm hot} - P_{\rm cold}}{T_{\rm hot} - T_{\rm cold}} $$

and the intercept is

$$ b = \frac{P_{\rm cold}T_{\rm hot} - P_{\rm hot}T_{\rm cold}}{T_{\rm hot} - T_{\rm cold}} $$

**Using these relations, compute the best-fit line and overplot it over the data.**

In [ ]:
# best-fit line
a_best = ...
b_best = ...

# plot power vs temperature with errors
fig = plt.figure(figsize=(10, 10))
plt.errorbar(...)
plt.plot(...)  # best-fit line
plt.tight_layout()

How does this compare to the results you got by doing "chi by eye"? Are you surprised?

# Naive Error Propagation

The "simplest" way to propagate errors is to essentially engage in a really simple thought experiment. Let's say that $P_{\rm hot}$ changed by a tiny amount $\Delta P_{\rm hot}$. That would shift $a$ by a corresponding amount $\Delta a$. Making $\Delta$ infinitely small implies that this difference should depend on the derivative of $a$ with respect to $P_{\rm hot}$. Likewise for $P_{\rm cold}$. We can formalize this with the basic result:

$$ \sigma_f^2 = \left|\frac{\partial f}{\partial x} \sigma_x\right|^2 + \left|\frac{\partial f}{\partial x} \sigma_y\right|^2 + \dots $$

In other words, the errors $\sigma_f$ on $f$ are just related to the errors on the individual parameters $x, y, \dots$ multiplied by how sensitive $f$ is to each of them.

**Using this result, compute the errors on $a$ and $b$ based on the measured errors in $P_{\rm hot}$ and $P_{\rm cold}$.**

In [ ]:
# derivatives
da_dp_hot = ...
da_dp_cold = ...
db_dp_hot = ...
db_dp_cold = ...

# errors in slope
a_err_naive = 

# errors in intercept
b_err_naive = 

Now, generate 10 realizations of the fitted line based on these naive errors and overplot them over the data. What do you see? Do these uncertainties accurately capture the expected behavior? Why or why not?

In [ ]:
# plot power vs temperature with errors
fig = plt.figure(figsize=(10, 10))
plt.errorbar(...)
plt.plot(...)  # best-fit line
for i in range(10):
    plt.plot(...)  # realizations of fit
plt.tight_layout()

Another way to look at this is to see the 2-D distribution of slopes vs intercepts. **Simulate 10000 slopes/intercepts from the naive errors and plot their distribution using `plt.hist2d`.** What do you see?

In [ ]:
# plot slope vs intercept distribution
plt.figure(figsize=(12, 10))
a_draws = ...
b_draws = ...
plt.hist2d(...)
plt.colorbar(label=...)  # add colorbar
...  # add axis labels and title
plt.tight_layout()

# Numerical Simulation

While the above is a great way to take a stab at the uncertainties, the real quantification of uncertainties can be substantially more complicated. For instance, we've measured the uncertainties here assuming that the mean and error that we've derived from our small set of observations accurately characterizes the true underlying measurement noise in the power from the receiver. This seems, at best, a stretch. Maybe it'd be a bit less of a stretch if we have many more observations though.

There are also other possible errors that could impact our analysis. Maybe there's **systematic errors** (i.e. something we're doing consistently wrong) with the way we're measuring the temperature, which we're currently not taking into account. Or maybe there's some criteria we've imposed as part of our experiment that affects what measurements we take. While effects like these can sometimes be hard to include in simple analytic models like the one we specified above, they can naturally be explored from simulations.

## Without Sampling Error

We first want to run a simulation that ignores the impact of our sampling error and assumes that our measured means and standard deviations for the power is correct. We can then simulate the effect these have on our fitted relation directly by:


1. Simulating new values for $P_{\rm hot}$ and $P_{\rm cold}$.
2. Fitting a line based on the analytic relationship above.
3. Repeat 1-2 a large number of times.
4. Analyze the fitted slopes and intercepts.

Let's implement this below. **Simulate 10000 realizations of the best-fit line and save the resulting slopes/intercepts.**

In [ ]:
a_arr, b_arr = ...

for i in range(10000):
    # simulate data
    P_hot_r, P_cold_r = np.random.normal(...), np.random.normal(...)
    
    # fit line
    a_r, b_r = ...
    
    # save fit
    a_arr[i], b_arr[i] = a_r, b_r

How do these compare to the values derived from our naive error analysis? **Plot the corresponding 2-D distributions for both analyses below.**

In [ ]:
# compare simple errors vs numerical simulation
plt.figure(figsize=(24, 10))

# naive errors
plt.subplot(1, 2, 1)
plt.hist2d(...)  # slope vs intercept
plt.colorbar(label=...)  # add colorbar
...  # add axis labels and title

# numerical simulation
plt.subplot(1, 2, 2)
plt.hist2d(...)  # slope vs intercept
plt.colorbar(label=...)  # add colorbar
...  # add axis labels and title

plt.tight_layout()

## With Sampling Error

Now we want to include the effect of sampling error, i.e. that we estimate the mean and variance from a small number of points. Here, let's assume that the means and errors we derive are correct, and analyze how following our analysis "end to end" introduces additional errors. This procedure now looks like:

1. Simulate $n$ new values for $P_{\rm hot}$ and $P_{\rm cold}$.
2. Compute the mean and standard deviation from the corresponding samples. This is analagous to what we measured.
3. Simulate a *new* value for $P_{\rm hot}$ and $P_{\rm cold}$ from the mean and standard deviation derived above (instead of the one we started with).
4. Fit a line based on the analytic relationship above.
5. Repeat 1-4 a large number of times.
6. Analyze the fitted slopes and intercepts.

Let's implement this below. **Simulate 10000 realizations of the best-fit line incorporating sampling uncertainty and save the resulting slopes/intercepts.**

In [ ]:
a_arr2, b_arr2 = ...

for i in range(10000):
    # simulate n datapoints
    P_hot_rs, P_cold_rs = np.random.normal(...), np.random.normal(...)
    
    # compute sample mean and standard deviation
    P_hot_mean_r, P_cold_mean_r = ..., ...
    P_hot_std_r, P_cold_std_r = ..., ...
    
    # simulate new datapoint
    P_hot_r, P_cold_r = np.random.normal(...), np.random.normal(...)

    
    # fit line with `P_hot_r` and `P_cold_r`
    a_r, b_r = ...
    
    # save fit
    a_arr2[i], b_arr2[i] = a_r, b_r

How do these compare to the values derived from our previous numerical simulation where we ignore this effect? **Plot the corresponding 2-D distributions below.**

In [ ]:
# compare numerical results w/ and w/o sampling noise
plt.figure(figsize=(24, 10))

# it's important to have the same bins
# so that the plots are to scale
bins = [np.linspace(a_best - 5 * a_err_naive, a_best + 5 * a_err_naive, 100),
        np.linspace(b_best - 5 * b_err_naive, b_best + 5 * b_err_naive, 100)]

# w/o sampling noise
# make sure to set `bins=bins` and `cmin=1` in `hist2d`
plt.subplot(1, 2, 1)
plt.hist2d(...)  # slope vs intercept
plt.colorbar(label=...)  # add colorbar
...  # add axis labels and title

# w/ sampling noise
# make sure to set `bins=bins` and `cmin=1` in `hist2d`
plt.subplot(1, 2, 2)
plt.hist2d(...)  # slope vs intercept
plt.colorbar(label=...)  # add colorbar
...  # add axis labels and title

plt.tight_layout()

These *look* pretty similar, but one way to quantify the differences is to compute the **covariance** among these realizations and see how they differ.

In [ ]:
# compute covariances
cov_wosamp = np.cov(np.c_[a_arr, b_arr], rowvar=False)
cov_wsamp = np.cov(np.c_[a_arr2, b_arr2], rowvar=False)

print('Without sampling noise:')
print(cov_wosamp)
print('With sampling noise:')
print(cov_wsamp)

Next time, we'll start exploring more detail about how to interpret these simulation results in a more rigorous statistical context.